**Installations:**

In [1]:
!pip install -r "/content/requirements.txt"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6


**Streamlit UI Development and Model Deployment:**

In [12]:
%%writefile app.py

# imports
import streamlit as st
from phi.agent import Agent
from phi.model.google import Gemini
from phi.tools.duckduckgo import DuckDuckGo
from google.generativeai import upload_file, get_file
import google.generativeai as genai
import time
from pathlib import Path
import tempfile
from dotenv import load_dotenv
load_dotenv()
import os

# Store the gemini api key
GOOGLE_API_KEY="YOUR GEMINI API KEY"

# Set Google API key
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

# Configure Google Gemini API
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])



# Page configuration
st.set_page_config(
    page_title='AI-Powered Video Analyzer',
    page_icon='🤖',
    layout='wide'
)

st.title("Smart Video Analyzer 🧠🎥")
st.header("Analyze, Understand, and Extract Insights from Videos — AI-Driven Video Intelligence at Your Fingertips.")

@st.cache_resource
# Initialize the AI agent
def initialize_agent():
  return Agent(
      name='AI Video Analyzer',
      model=Gemini(id="gemini-2.0-flash-exp"),
      tools=[DuckDuckGo()],
      markdown=True

  )

multimodal_Agent=initialize_agent()


# File uploader
video_file=st.file_uploader(
    "Upload a video file",
    type=["mp4","mov","avi"],
    help="Upload a video for AI analysis"


)

if video_file:
    with tempfile.NamedTemporaryFile(delete=False, suffix='.mp4') as temp_video:
      temp_video.write(video_file.read())
      video_path=temp_video.name

    st.video(video_path, format='video/mp4', start_time=0)

    user_query=st.text_area(
      "What insights are you seeking from the video?",
      placeholder='Ask anything about the video content. The AI agent will analyze and gather additional context if needed.',
      help="Provide specific questions or insights you want from the video."
   )

    if st.button("🔍 Analyze Video", key="analyze_video_button"):

      if not user_query:
        st.warning("Please enter a question or ask for insights.")
      else:

        try:
          with st.spinner("Processing video and gathering insights..."):
            # Upload and process video file
            processed_video= upload_file(video_path)
            while processed_video.state.name=='PROCESSING':
              time.sleep(1)
              processed_video=get_file(processed_video.name)


            # Prompt generation for analysis
            analysis_prompt=(
                f"""
                Analyze the uploaded video for content and context.
                Respond to the following query using video insights and supplementary web research:
                {user_query}

                Provide a detailed, user-friendly, and actionable response.
                """
            )

            # AI agent processing
            response= multimodal_Agent.run(analysis_prompt, videos=[processed_video])

          # Display the result
          st.subheader("Analysis Result")
          st.markdown(response.content)

        except Exception as error:
          st.error(f"An error occurred during analysis: {error}")
        finally:
          # Clean up temporary video file
          Path(video_path).unlink(missing_ok=True)


else:
  st.info("Upload a video file to begin analysis.")


# Customize text area height
st.markdown(
    """
    <style>
    .stTextArea textarea {
        height: 100px;
    }
    </style>
    """,
    unsafe_allow_html=True
)




Overwriting app.py


**Install LocalTunnel:**

In [4]:
!npm install localtunnel


⠙⠹⠸⠼⠴⠦⠧⠇⠏
up to date, audited 23 packages in 2s
⠋
⠋3 packages are looking for funding
⠋  run `npm fund` for details
⠋
2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠋

**Get the password for the tunnel:**

In [5]:
!wget -q -O - ipv4.icanhazip.com  # This will give us the password for local tunnel

35.201.177.3


**Run the Streamlit application:**

In [6]:
!streamlit run app.py & npx localtunnel --port 8501 # This allows us to view our Streamlit application in the web browser



⠙⠹
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.201.177.3:8501

your url is: https://cyan-sites-wink.loca.lt
  Stopping...
^C
